In [ ]:
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
import math

from PIL import Image, ImageStat
from os.path import exists

In [ ]:
def get_icon_from_item(item):
    filename = path_grid_icons + item['id'] + filename_ending_grid_icon
    if exists(filename):
        return np.asarray(Image.open(filename))
    else:
        print(f"File {filename} does not exist.")
    return []

def get_rand_item_slot():
    global all_items_df
    
    icon = []
    while len(icon) == 0:
        index = rand.randrange(len(all_items_df)) # TODO change
        item = all_items_df.iloc[index]
        icon = get_icon_from_item(item)
    
    w_index = rand.randrange(item['width'])
    h_index = rand.randrange(item['height'])
    slot_width = (int) (item['icon_width'])
    slot_height = (int) (item['icon_height'])
    
    subicon = icon[h_index*slot_height:(h_index+1)*slot_height, w_index*slot_width:(w_index+1)*slot_width]
    return index,subicon
    
def get_feature_from_img(img):
    width = img.shape[1]
    height = img.shape[0]
    r = 0;
    g = 0;
    b = 0;
    for x in range(width):
        for y in range(height):
            r += (icon[y][x])[0]
            g += (icon[y][x])[1]
            b += (icon[y][x])[2]
    r = (int) (r / (width*height))
    g = (int) (g / (width*height))
    b = (int) (b / (width*height))
    return (r,g,b)

In [ ]:
# paths
path_icons = './icons/'
path_data = './data/'
path_grid_icons = './grid_icons/'

filename_ending_grid_icon = '-grid-image.jpg'

In [ ]:
# load data from csv
sigma = 20
all_items_df = pd.read_csv(path_data + 'all_items.csv')
feature_dict_r = pd.read_csv(path_data + 'all_features_r_sigma_' + str(sigma) + '.csv')
feature_dict_g = pd.read_csv(path_data + 'all_features_g_sigma_' + str(sigma) + '.csv')
feature_dict_b = pd.read_csv(path_data + 'all_features_b_sigma_' + str(sigma) + '.csv')

In [ ]:
# get random icon
true_index,icon = get_rand_item_slot()
feature = get_feature_from_img(icon)

# create voting vector
voting = []
for i in range(len(all_items_df)):
    voting.append(0)
    
# vote
sigma = 1
max_index = len(all_items_df)-1
# for c in feature:
#     print(c)
li = (feature_dict_r.iloc[feature[0]]['features']).strip('][').split(', ')
for index in li:
    for i in range(-sigma,sigma,1):
        if int(index)+i < 0 or int(index)+i > max_index:
            continue
        voting[int(index)+i] += 1
li = (feature_dict_g.iloc[feature[1]]['features']).strip('][').split(', ')
for index in li:
    for i in range(-sigma,sigma,1):
        if int(index)+i < 0 or int(index)+i > max_index:
            continue
        voting[int(index)+i] += 1
li = (feature_dict_b.iloc[feature[2]]['features']).strip('][').split(', ')
for index in li:
    for i in range(-sigma,sigma,1):
        if int(index)+i < 0 or int(index)+i > max_index:
            continue
        voting[int(index)+i] += 1

In [ ]:
# li

In [ ]:
Image.fromarray(icon)

In [ ]:
# feature_dict.iloc[100]['features']

In [ ]:
indexes = []
value = 0
total = 0
# find max value
for i in range(len(voting)):
    if voting[i] > value:
        value = voting[i]
        index = i
# find all indexes with max value
for i in range(len(voting)):
    if voting[i] == value:
        total += 1
        indexes.append(i)
        
print(f"Predicted index {index} with voting {value}, total {total}")
print(f"True index is {true_index}")
print(f"Indexes: {indexes}")

In [ ]:
w = 10
h = 10
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = math.ceil(len(indexes)/4)
for i in range(len(indexes)):
    img = get_icon_from_item(all_items_df.iloc[indexes[i]])
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)
plt.show()